In [4]:
from __future__ import division
from numpy import pi, linspace, sin, cos, exp, log, sqrt, dot
import numpy.random as rnd
import numpy as np
import matplotlib.pyplot as plt
#import theano as T

%matplotlib inline

sigmoid = lambda x: 1/(1+np.exp(-x))

for 2-way probabilistic classification: sigmoid non-linearity $(r(a)={\rm sigmoid}(a)=1/(1+e^{-a})$, applied element by element), and cross-entropy loss $L(f,(x,y))= -y \log f(x) -(1-y)\log(1-f(x))$ for y binary. Note that the sigmoid output $ f(x) $ is in the (0,1) interval, and corresponds to an estimator of $ P(y=1|x) $. The predicted class is 1 if $f(x)>\frac{1}{2}.$

<h1> Copy from .py file </h1>

In [181]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))


In [177]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [182]:
net = Network([784, 30,10])
net.SGD(training_data, 15, 10, 3.0, test_data=test_data)

Epoch 0: 9145 / 10000
Epoch 1: 9236 / 10000


KeyboardInterrupt: 

<h1> Write own neural network code </h1>

In [195]:
""" Multi Layer Perceptron"""


class multilayer_perceptron(object):
    
    
    def __init__(self,units_per_layer,activation='sigmoid'):
        self.num_layers = len(units_per_layer)
        self.biases = [np.random.randn(y, 1) for y in units_per_layer[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(units_per_layer[:-1], units_per_layer[1:])]

        if activation=='sigmoid':
            self.activation = lambda x: 1./(1.+np.exp(-x))
            self.activation_deriv = lambda x: sigmoid(x)*(1-sigmoid(x))
        else:
            raise ValueError



    def forwardpass(self, a):
        """Return the output of the network if 'a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = self.activation(np.dot(w, a)+b)
        return a


   
            
    def backprop(self,network_input,target_activation):

            delta_cost_biass = [np.zeros(b.shape) for b in self.biases]
            delta_cost_weights = [np.zeros(w.shape) for w in self.weights]

            weighted_activations = []; 
            a = network_input
            activations = [a]
            "first perform the forward pass"
            for b, w in zip(self.biases, self.weights):
                #print 'a: %s, b: %s, c: %s' %(a.shape, b.shape, w.shape)
                z = np.dot(w, a)+b 
                weighted_activations.append(z)
                a = sigmoid(z)
                activations.append(a)

            "calculate the ouput errors"
            delta = (activations[-1]-target_activation)*self.activation_deriv(weighted_activations[-1])
            #delta = self.get_output_error(activations[-1],target_activation,weighted_activations[-1])
            delta_cost_biass[-1] = delta
            delta_cost_weights[-1] = np.dot(delta, activations[-2].transpose())
            "propagate the errors back"
            #layer_errors = []
            #layer_errors[-1] = delta
            for l in xrange(2,self.num_layers):
                delta = np.dot(self.weights[-l+1].T,delta)*self.activation_deriv(weighted_activations[-l])
                delta_cost_biass[-l] = delta
                delta_cost_weights[-l] =  np.dot(delta, activations[-l-1].T)

            return delta_cost_weights, delta_cost_biass
        
    
    def minibatch_weight_update(self,mini_batch,eta):

        delta_cost_biass = [np.zeros(b.shape) for b in self.biases]
        delta_cost_weights = [np.zeros(w.shape) for w in self.weights]
        
        #recursive for calculating errors over training examples
        for x, y in mini_batch:
            delta_w, delta_b = self.backprop(x, y)
            delta_cost_biass = [nb+dnb for nb, dnb in zip(delta_cost_biass, delta_b)]
            delta_cost_weights = [nw+dnw for nw, dnw in zip(delta_cost_weights, delta_w)]
        # NOTE: eta here is the learning rate. Update weights according to
        # rule of 
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, delta_cost_weights)]
        #print b.shape, nb.shape
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, delta_cost_biass)]
    
    def stochastic_gradient_descent(self, training_data, epochs, mini_batch_size, eta,test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                #print [i.shape for i in self.biases]
                self.minibatch_weight_update(mini_batch, eta)
                #print 'ho'
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
    
    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.forwardpass(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    
                
                
                

In [196]:
mlp = multilayer_perceptron([784, 40, 10])
mlp.stochastic_gradient_descent(training_data, 15, 10, 3.0, test_data=test_data)

Epoch 0: 9025 / 10000
Epoch 1: 9327 / 10000
Epoch 2: 9367 / 10000
Epoch 3: 9363 / 10000


KeyboardInterrupt: 

In [ ]:
def backprop_2(self,network_input,target_activation):
        
        delta_cost_biass = [np.zeros(b.shape) for b in self.biases]
        delta_cost_weights = [np.zeros(w.shape) for w in self.weights]
        
        weighted_activations = []; 
        a = network_input
        activations = [a]
        "first perform the forward pass"
        for b, w in zip(self.biases, self.weights):
            #print 'a: %s, b: %s, c: %s' %(a.shape, b.shape, w.shape)
            z = np.dot(w, a)+b 
            weighted_activations.append(z)
            a = sigmoid(z)
            activations.append(a)
        
        "calculate the ouput errors"
        delta = (activations[-1]-target_activation)*sigmoid_prime(weighted_activations[-1])
        #delta = self.get_output_error(activations[-1],target_activation,weighted_activations[-1])
        delta_cost_biass[-1] = delta
        delta_cost_weights[-1] = np.dot(delta, activations[-2].transpose())


        "propagate the errors back"
        #layer_errors = []
        #layer_errors[-1] = delta
        for l in xrange(2,self.num_layers):
            delta = np.dot(self.weights[-l+1].T,delta)*sigmoid_prime(weighted_activations[-l])
            delta_cost_biass[-l] = delta
            delta_cost_weights[-l] =  np.dot(delta, activations[-l-1].T)
        
        return delta_cost_biass, delta_cost_weights

In [ ]:
  def backprop(self, x, y):
        """Return a tuple '(nabla_b, nabla_w)' representing the
        gradient for the cost function C_x.  'nabla_b' and
        'nabla_w' are layer-by-layer lists of numpy arrays, similar
        to 'self.biases' and 'self.weights'."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer. z is the weighted input to each neuron
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b 
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            

        # backward pass
        
        """Here calculate the difference between the actual activations
           in the !!!final layer!!! and the target and the derivative of the 
           activation function at the activation point.
        """
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        
        "Here propagate the error back back through layers preceeding the final one"
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)




<h1> How to draw an elephant </h1>

"With four free parameters I can fit you an elephant" - John von Neumann

In [106]:
"""
Author: Piotr A. Zolnierczuk (zolnierczukp at ornl dot gov)
 
Based on a paper by:
Drawing an elephant with four complex parameters
Jurgen Mayer, Khaled Khairy, and Jonathon Howard,
Am. J. Phys. 78, 648 (2010), DOI:10.1119/1.3254017
"""
import numpy as np
import pylab
 
# elephant parameters
p1, p2, p3, p4 = (50 - 30j, 18 +  8j, 12 - 10j, -14 - 60j )
p5 = 40 + 20j # eyepiece
 
def fourier(t, C):
    f = np.zeros(t.shape)
    A, B = C.real, C.imag
    for k in range(len(C)):
        f = f + A[k]*np.cos(k*t) + B[k]*np.sin(k*t)
    return f
 
def elephant(t, p1, p2, p3, p4, p5):
    npar = 6
    Cx = np.zeros((npar,), dtype='complex')
    Cy = np.zeros((npar,), dtype='complex')
 
    Cx[1] = p1.real*1j
    Cx[2] = p2.real*1j
    Cx[3] = p3.real
    Cx[5] = p4.real
 
    Cy[1] = p4.imag + p1.imag*1j
    Cy[2] = p2.imag*1j
    Cy[3] = p3.imag*1j
 
    x = np.append(fourier(t,Cx), [-p5.imag])
    y = np.append(fourier(t,Cy), [p5.imag])
 
    return x,y
 
pylab.figure(figsize=(2,2))
x, y = elephant(np.linspace(0,2*np.pi,1000), p1, p2, p3, p4, p5)
pylab.plot(y,-x,'.')
pylab.show()
